In [1]:
%matplotlib inline

# Silence warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from keras import utils
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, MaxPool2D
from keras import Sequential, optimizers
from keras.optimizers import Adam
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input, decode_predictions, VGG16


Using TensorFlow backend.


In [ ]:
#Prepare the data
(X_train, y_train), (X_test,y_test)=fashion_mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype(np.float64)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype(np.float64)
X_train=X_train/255.
X_test=X_test/255.

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=10000,
                                                  random_state=42)

num_classes = 10
y_train_ = utils.to_categorical(y_train, num_classes)
y_val = utils.to_categorical(y_val, num_classes)
y_test = utils.to_categorical(y_test, num_classes)
y_train = utils.to_categorical(y_train, num_classes)

In [ ]:
sgd = optimizers.SGD(lr=0.03, momentum=0.9, nesterov=True)

x=Input((28,28,1))
h=Conv2D(10, (3,3), activation='relu')(x)
h=MaxPool2D(2, strides=1)(h)

h=Flatten()(h)
h=Dense(15, activation='relu')(h)
h=Dense(15, activation='relu')(h)
h=Dense(10, activation='softmax')(h)
conv=Model(x,h)
conv.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
conv.fit(X_train, y_train, epochs=50,batch_size=128, validation_data=(X_val, y_val))

In [ ]:

# load the model
model = VGG16(include_top=True,weights='imagenet')


In [ ]:
# load an image from file
testimage = load_img('fruits.jpg', target_size=(224, 224))
testimage = img_to_array(testimage)
plt.imshow(testimage/testimage.max())
plt.axis('off')
testimage = testimage.reshape((1, testimage.shape[0], testimage.shape[1], testimage.shape[2]))
testimage = preprocess_input(testimage)
prediction = model.predict(testimage)
label = decode_predictions(prediction)
print(label)

In [36]:
#Load dataset. mountain bikes=[1,0], road bikes=[0,1]
X=[]
y=[]
for i in range(105):
    path='bikes\mountain_bikes\mountain_bike_'+str(i)+'.jpg'

    image = load_img(path, target_size=(224, 224))
    image = img_to_array(image)
    #image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    #image=np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    X.append(image)
    #y.append([1,0])
    y.append(1)
    
for j in range(106):
    path='bikes\\road_bikes\\road_bike_'+str(j)+'.jpg'

    #path='bikes\road_bikes\road_bike_'+str(j)+'.jpg'
    image = load_img(path, target_size=(224, 224))
    image = img_to_array(image)
    #image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    #image=np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    X.append(image)
    #y.append([0,1])
    y.append(0)

X=np.array(X)
#y=np.array(y)
y=to_categorical(y,2)


In [38]:
X_trainval, X_test, y_trainval, y_test=train_test_split(X,y, train_size=0.7, random_state=0)
X_train, X_val,y_train, y_val=train_test_split(X_trainval,y_trainval, train_size=0.7, random_state=0)


In [40]:


vgg16 = VGG16(weights='imagenet', include_top=False)
vgg16.summary()

inp = Input((224,224,3,))

output_vgg16_conv = vgg16(inp)

x = Flatten()(output_vgg16_conv)
x=Dense(30, activation='relu')(x)
x = Dense(2, activation='softmax')(x)

bikemodel = Model(inp,x)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [ ]:
sgd = optimizers.SGD(lr=0.01)

bikemodel.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bikemodel.fit(X_train, y_train, epochs=140,validation_data=(X_val, y_val))

Train on 102 samples, validate on 45 samples
Epoch 1/140
102/102 [==============================] - 142s 1s/step - loss: 7.1850 - acc: 0.5490 - val_loss: 6.8054 - val_acc: 0.5778
Epoch 2/140
102/102 [==============================] - 141s 1s/step - loss: 8.8492 - acc: 0.4510 - val_loss: 6.8054 - val_acc: 0.5778
Epoch 3/140
102/102 [==============================] - 141s 1s/step - loss: 8.8492 - acc: 0.4510 - val_loss: 6.8054 - val_acc: 0.5778
Epoch 4/140
102/102 [==============================] - 141s 1s/step - loss: 8.8492 - acc: 0.4510 - val_loss: 6.8054 - val_acc: 0.5778
Epoch 5/140
102/102 [==============================] - 141s 1s/step - loss: 8.8492 - acc: 0.4510 - val_loss: 6.8054 - val_acc: 0.5778
Epoch 6/140
 32/102 [========>.....................] - ETA: 1:23 - loss: 8.5627 - acc: 0.4688

In [ ]:
def d(inp):
    y=inp[np.newaxis,...]
    return y

def whichbike(arr):
    mtb_proba=arr[0]
    road_proba=[1]
    print("Mountain bike: %f \n Road bike: %g" %(mtb_proba, road_proba))

In [27]:
bikemodel.predict(X_test)

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan,

In [28]:
y_test

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.